In [1]:
import pandas as pd
from soil_fertility.components.data_ingestion import DataIngestion
from soil_fertility.components.data_transformation.data_transformation import DataTransformation
from soil_fertility.components.data_transformation.data_transformation_part3 import DataTransformationThree
from soil_fertility.components.data_transformation.data_transformation_part2 import DataTransformationTwo
import warnings
warnings.filterwarnings('ignore')

In [2]:
obj = DataIngestion()
part,train_path,test_path= obj.init_ingestion(
    "/home/redha/Documents/projects/NLP/datamining project/Soil-Fertility/data/Dataset1.csv"
)
part2,train_path2,test_path2= obj.init_ingestion(
    "/home/redha/Documents/projects/NLP/datamining project/Soil-Fertility/data/Dataset2.csv"
)
part3,train_path3,test_path3= obj.init_ingestion(
    "/home/redha/Documents/projects/NLP/datamining project/Soil-Fertility/data/Dataset3.xlsx",option="xlsx"
)
data_path=r'/home/redha/Documents/projects/NLP/datamining project/Soil-Fertility/data/Dataset2.csv'


In [4]:
transformation_obj=DataTransformationTwo()
part,preprocessor,data_path=transformation_obj.transform(data_path)

In [5]:
s=pd.read_csv(data_path)

In [6]:
tranformation_obj = DataTransformation()
part,preprocessor,processed_train_path,processed_test_path = tranformation_obj.transform(
    train_path,test_path,target="Fertility"
)

In [7]:
tranformation_obj = DataTransformationThree()
part,preprocessor,processed_train_path,processed_test_path = tranformation_obj.transform(
    train_path3,test_path3,numerical_features=["Temperature","Humidity","Rainfall"],k=5,strategie="frequency"
)

In [8]:
df=pd.read_csv(processed_train_path)

In [ ]:
from soil_fertility.components.models import AssociationRuleMiner
obj = AssociationRuleMiner()
obj.execute(df,transactions_groups=["Soil"],items_groups=["Crop","Fertilizer","Temperature"	,"Humidity"	,"Rainfall"])